In [ ]:
%pip install pandas nltk scikit-learn spacy
%python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd

# Load your DataFrame
df = pd.read_excel('linkedin_jobs.xlsx')



In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Combine NLTK and sklearn stop words
stop_words = set(stopwords.words('english')).union(set(stopwords.words('german')), set(stopwords.words('french')), ENGLISH_STOP_WORDS)

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['Description'] = df['Description'].apply(preprocess_text)


In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

df['Description'] = df['Description'].apply(lemmatize_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(df['Description'])

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Sum the TF-IDF values for each term across all documents
sum_tfidf = X.sum(axis=0)
tfidf_scores = [(word, sum_tfidf[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

# Print top 20 keywords
for word, score in tfidf_scores[:50]:
    print(f"{word}: {score:.4f}")
